In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44797
2,Huelva,Confirmados PDIA 14 días,658
3,Huelva,Tasa PDIA 14 días,"128,22261628700042"
4,Huelva,Confirmados PDIA 7 días,229
5,Huelva,Tasa PDIA 7 dias,"44,624588343044216"
6,Huelva,Total Confirmados,44996
7,Huelva,Curados,41319
8,Huelva,Fallecidos,407


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44797.0


/tmp/ipykernel_3751/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12409.0


/tmp/ipykernel_3751/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3751/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3751/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3751/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3196 personas en los últimos 7 días 

Un positivo PCR cada 1035 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44797.0,229.0,658.0,513170.0,44.624588,128.222616,18.0
Huelva-Costa,26494.0,163.0,427.0,289548.0,56.294639,147.471231,14.0
Cartaya,1906.0,37.0,71.0,20083.0,184.235423,353.532839,5.0
Huelva (capital),12409.0,45.0,139.0,143837.0,31.285413,96.637166,4.0
Condado-Campiña,13919.0,42.0,163.0,156231.0,26.883269,104.332687,2.0
Isla Cristina,3082.0,26.0,62.0,21393.0,121.535082,289.814425,2.0
Lepe,2971.0,15.0,39.0,27880.0,53.802009,139.885222,2.0
San Juan del Puerto,757.0,5.0,9.0,9411.0,53.129317,95.632770,2.0
Moguer,1928.0,7.0,27.0,21867.0,32.011707,123.473728,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,32.0,1.0,5.0,396.0,252.525253,1262.626263,0.0
Villablanca,284.0,7.0,15.0,2885.0,242.634315,519.930676,0.0
Higuera de la Sierra,64.0,2.0,6.0,1291.0,154.918668,464.756003,0.0
San Bartolomé de la Torre,376.0,1.0,17.0,3761.0,26.588673,452.007445,0.0
Aroche,126.0,7.0,13.0,3054.0,229.207597,425.671251,0.0
Cartaya,1906.0,37.0,71.0,20083.0,184.235423,353.532839,5.0
Puebla de Guzmán,174.0,3.0,9.0,3092.0,97.024580,291.073739,0.0
Isla Cristina,3082.0,26.0,62.0,21393.0,121.535082,289.814425,2.0
Ayamonte,2015.0,21.0,45.0,21104.0,99.507202,213.229719,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
San Bartolomé de la Torre,376.0,1.0,17.0,3761.0,26.588673,452.007445,0.0,0.058824
Gibraleón,923.0,1.0,15.0,12737.0,7.851142,117.767135,-1.0,0.066667
Aljaraque,1609.0,1.0,12.0,21474.0,4.656794,55.881531,0.0,0.083333
Palma del Condado (La),1414.0,2.0,12.0,10801.0,18.516804,111.100824,0.0,0.166667
Rociana del Condado,660.0,1.0,5.0,7939.0,12.596045,62.980224,0.0,0.200000
Cañaveral de León,32.0,1.0,5.0,396.0,252.525253,1262.626263,0.0,0.200000
Condado-Campiña,13919.0,42.0,163.0,156231.0,26.883269,104.332687,2.0,0.257669
Moguer,1928.0,7.0,27.0,21867.0,32.011707,123.473728,1.0,0.259259
Punta Umbría,1253.0,5.0,17.0,15355.0,32.562683,110.713123,1.0,0.294118
